In [1]:
# Dataset management packages
from spivutils.synthetic_datasets.spid import load_data
from spivutils.batch_generators.keras_generator import batch_data
from spivutils.common_tools.operations import normalization, vectoraddition, thresholding

# General purpose packages
import numpy as np

# Model packages
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Conv2D, Conv3D, MaxPooling3D, MaxPooling2D,ConvLSTM2D,Flatten, Dense, Reshape, Activation, LSTM, TimeDistributed

2023-10-14 13:17:00.286087: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-14 13:17:00.910004: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
# Detects the avaiable hardware
cpu_devices = tf.config.list_physical_devices('CPU')
gpu_devices = tf.config.experimental.list_physical_devices('GPU')

# Allow memory growth
for gpu in gpu_devices:
  tf.config.experimental.set_memory_growth(gpu, True)

2023-10-14 13:17:01.767673: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-10-14 13:17:01.791702: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-10-14 13:17:01.791959: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.


In [3]:
# Data importing
(train_x, train_y), (valid_x, valid_y), (test_x, test_y) = load_data()


 Checking files ...

 train_x.npy  Found!

 train_y.npy  Found!

 valid_x.npy  Found!

 valid_y.npy  Found!

 test_x.npy  Found!

 test_y.npy  Found!

 Importing data ...

 Data loading successful!


In [4]:
batch_size = 5
chunk_size = 100

# Collect a chunk of the dataset
train_x_chunk = train_x[0:chunk_size]
train_y_chunk = train_y[0:chunk_size]
valid_x_chunk = valid_x[0:chunk_size]
valid_y_chunk = valid_y[0:chunk_size]

# Load data in batches to avoid memory overload
train_batch = batch_data(train_x_chunk, train_y_chunk, batch_size)
valid_batch = batch_data(valid_x_chunk, valid_y_chunk, batch_size)

In [5]:
# Add pre-processing operations
train_batch.add_x_preprocessing_operation(thresholding)
train_batch.add_x_preprocessing_operation(normalization)
train_batch.add_y_preprocessing_operation(vectoraddition)

valid_batch.add_x_preprocessing_operation(thresholding)
valid_batch.add_x_preprocessing_operation(normalization)
valid_batch.add_y_preprocessing_operation(vectoraddition)

In [6]:
input_shape = train_batch[0][0][0,].shape
output_shape = train_batch[0][1][0,].shape
input_shape_time = (2,) + input_shape[2:]

# Model hyperparameters
filters = 32
kernel_size = (10,10)
pool_size = (5 ,5)
units = 32

In [7]:
model = Sequential()
model.add(ConvLSTM2D(filters=filters, kernel_size=kernel_size, padding='same', input_shape=input_shape, return_sequences=False))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=pool_size))
model.add(Flatten())
model.add(Dense(units))
model.add(Activation('relu'))
model.add(Dense(np.prod(output_shape)))
model.add(Activation('linear'))
model.add(Reshape(output_shape))
model.compile(loss='mean_squared_error', optimizer='adam')

2023-10-14 13:17:02.676909: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-10-14 13:17:02.677265: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-10-14 13:17:02.677501: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-10-14 13:17:03.559140: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-10-14 13:17:03.559412: I tensorflow/compile

In [8]:
model.build((None,) + input_shape)

In [9]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv_lstm2d (ConvLSTM2D)    (None, 630, 665, 32)      422528    
                                                                 
 activation (Activation)     (None, 630, 665, 32)      0         
                                                                 
 max_pooling2d (MaxPooling2  (None, 126, 133, 32)      0         
 D)                                                              
                                                                 
 flatten (Flatten)           (None, 536256)            0         
                                                                 
 dense (Dense)               (None, 32)                17160224  
                                                                 
 activation_1 (Activation)   (None, 32)                0         
                                                        

In [10]:
model.fit(train_batch, validation_data = valid_batch, epochs = 1)

2023-10-14 13:17:07.464535: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8600
2023-10-14 13:17:22.368607: W tensorflow/tsl/framework/bfc_allocator.cc:485] Allocator (GPU_0_bfc) ran out of memory trying to allocate 256.50MiB (rounded to 268957440)requested by op sequential/conv_lstm2d/while/body/_1/sequential/conv_lstm2d/while/convolution_6
If the cause is memory fragmentation maybe the environment variable 'TF_GPU_ALLOCATOR=cuda_malloc_async' will improve the situation. 
Current allocation summary follows.
Current allocation summary follows.
2023-10-14 13:17:22.368768: I tensorflow/tsl/framework/bfc_allocator.cc:1039] BFCAllocator dump for GPU_0_bfc
2023-10-14 13:17:22.368818: I tensorflow/tsl/framework/bfc_allocator.cc:1046] Bin (256): 	Total Chunks: 34, Chunks in use: 34. 8.5KiB allocated for chunks. 8.5KiB in use in bin. 528B client-requested in use in bin.
2023-10-14 13:17:22.368843: I tensorflow/tsl/framework/bfc_allocator.cc:1046] Bin (512)

ResourceExhaustedError: Graph execution error:

Detected at node 'sequential/conv_lstm2d/while/convolution_6' defined at (most recent call last):
    File "/home/michel/miniconda3/envs/tf/lib/python3.9/runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/home/michel/miniconda3/envs/tf/lib/python3.9/runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "/home/michel/miniconda3/envs/tf/lib/python3.9/site-packages/ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "/home/michel/miniconda3/envs/tf/lib/python3.9/site-packages/traitlets/config/application.py", line 1046, in launch_instance
      app.start()
    File "/home/michel/miniconda3/envs/tf/lib/python3.9/site-packages/ipykernel/kernelapp.py", line 736, in start
      self.io_loop.start()
    File "/home/michel/miniconda3/envs/tf/lib/python3.9/site-packages/tornado/platform/asyncio.py", line 199, in start
      self.asyncio_loop.run